## UAV-Cyber-SIM

In [2]:
import random
from collections import defaultdict

from config import Color
from helpers import clean
from mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()

## Create Plans

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs=[Color.RED,Color.ORANGE, Color.GREEN,Color.BLUE]
n_uavs_per_gcs = 25
side_len = 10
altitude = 5
max_delay = 10 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids: dict[str, list[int]] = defaultdict(list)

for i,color in enumerate(colors,start=1):
    gcs_sysids[f'{color.name} {color.emoji}'].append(i)

## Gazebo Congiguration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                          world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

# No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)

## Visualization Parameters

In [6]:
novis=NoVisualizer(novis_config)

gaz= Gazebo(gaz_config,gra_origin)

qgc=QGC(qgc_config)

## Launch Simulator

In [7]:
simulator = Simulator(
	visualizers=[qgc],
	gcs_sysids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

🚀 ArduPilot SITL vehicle 4 launched (PID 1131416)
🚀 ArduPilot SITL vehicle 2 launched (PID 1131412)
🚀 ArduPilot SITL vehicle 1 launched (PID 1131411)
🚀 ArduPilot SITL vehicle 3 launched (PID 1131415)
🚀 ArduPilot SITL vehicle 6 launched (PID 1131421)
🚀 ArduPilot SITL vehicle 8 launched (PID 1131423)
🚀 ArduPilot SITL vehicle 5 launched (PID 1131418)
🚀 UAV logic for vehicle 2 launched (PID 1131425)
🚀 UAV logic for vehicle 4 launched (PID 1131424)
🚀 ArduPilot SITL vehicle 7 launched (PID 1131420)
🚀 UAV logic for vehicle 1 launched (PID 1131430)
🚀 UAV logic for vehicle 6 launched (PID 1131436)
🚀 UAV logic for vehicle 5 launched (PID 1131447)
🚀 UAV logic for vehicle 3 launched (PID 1131438)
🚀 ArduPilot SITL vehicle 9 launched (PID 1131428)
🚀 UAV logic for vehicle 8 launched (PID 1131442)
🚀 ArduPilot SITL vehicle 10 launched (PID 1131444)
🚀 Proxy for vehicle 2 launched (PID 1131449)
🚀 UAV logic for vehicle 7 launched (PID 1131452)
🚀 Proxy for vehicle 5 launched (PID 1131466)
🚀 Proxy for vehic

Vehicle 94: ✅ 8 waypoints read.


## Oracle checking

In [ ]:
while len(orac.conns):
    for sysid in list(orac.conns.keys()):
        if orac.is_plan_done(sysid):
            orac.remove(sysid)

Vehicle 95: ✅ 8 waypoints read.
Vehicle 93: ✅ 7 waypoints read.
Vehicle 77: ✅ 8 waypoints read.
Vehicle 84: ✅ 8 waypoints read.
Vehicle 89: ✅ 8 waypoints read.
Vehicle 85: ✅ 7 waypoints read.
Vehicle 76: ✅ 8 waypoints read.
Vehicle 97: ✅ 8 waypoints read.
Vehicle 66: ✅ Sent mission item 0
Vehicle 66: ✅ Sent mission item 1
Vehicle 66: ✅ Sent mission item 2
Vehicle 68: ✅ Sent mission item 0
Vehicle 66: ✅ Sent mission item 3
Vehicle 67: ✅ Sent mission item 0
Vehicle 68: ✅ Sent mission item 1
Vehicle 66: ✅ Sent mission item 4
Vehicle 67: ✅ Sent mission item 1
Vehicle 68: ✅ Sent mission item 2
Vehicle 66: ✅ Sent mission item 5
Vehicle 67: ✅ Sent mission item 2
Vehicle 68: ✅ Sent mission item 3
Vehicle 66: ✅ Sent mission item 6
Vehicle 67: ✅ Sent mission item 3
Vehicle 68: ✅ Sent mission item 4
Vehicle 66: ✅ Sent mission item 7
Vehicle 73: ✅ Sent mission item 0
Vehicle 67: ✅ Sent mission item 4
Vehicle 68: ✅ Sent mission item 5
Vehicle 66: ✅ Mission upload successful!
Vehicle 79: ✅ Sent miss